This Notebook launches a convergence study for the 1D Shock-Acoustic-Wave interaction problem as presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [ ]:
#r "BoSSSpad.dll"
#r "XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
Init();


In [ ]:
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D_ConvergenceStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase

In [ ]:
BoSSSshell.GetDefaultQueue()

In [ ]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

In [ ]:
BoSSSshell.WorkflowMgm.DefaultDatabase.Sessions

In [ ]:
BoSSSshell.WorkflowMgm.Sessions

In [ ]:
int[] Configs = new int[] {2};
int[] DGdegrees = new int[] {3};
int[] Grid_Res = new int[] {0,1,3,7};
double[] Amplitudes = new double[] {1e-5};
string[] WaveForms= new string[] {"1sinus"};
string[] LevelSetConfigs = new string[] {"true"};//,"false"};
double[] MachLs = new double[] {1.5};
double[] WLs = new double[] {0.8};

In [ ]:
//foreach(double iProb in iProbs) {
    double amp=1e-5;
    double amp_neg=0.0;
    double amp_pos=amp;
    double endTime=1.5;
    double shockPos=1.5;
    double wavePos=0.0;
    double scaling=1;
    double waveLength=0.4;
    double MachL=1.50;
foreach(double waveLength in WLs ){
foreach(double MachL in MachLs ){
foreach(double amp in Amplitudes ){
foreach(double config in Configs) {
foreach(string waveform in WaveForms) {
foreach(string withLS in LevelSetConfigs){
    amp_neg= 0.0;            amp_pos= amp;
    if(config==1){ //downstream negative acoustic wave (slow)
        amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;          shockPos=0.5;            wavePos=0.9;
    } else if(config==2){ //upstream positiv acoustic wave (fast)
        amp_neg= 0.0;            amp_pos= amp;            endTime=1.5;            shockPos=1.5;            wavePos=0.0;
    } else if(config==3){ //upstream negative acoustic wave (still fast)
        amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;           shockPos=1.5;            wavePos=0.3;
    }
    
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
            int cells=0;
            bool wLS;
            if(withLS == "true"){
                cells = (n+1)*15+1;wLS=true;
            }else{
                cells=(n+1)*15;wLS=false;
            }
                 var tmp_cntrl = XESTSF.XESTSFHardCodedControl.AcousticWave1D(waveLength: waveLength ,scaling: scaling ,endTime: endTime ,MachL: MachL ,withLevelSet: wLS ,wavePosition: wavePos , shockPosition: shockPos ,p_amp_neg:  amp_neg , p_amp_pos: amp_pos ,waveform: waveform,dgDegree: p ,numOfCellsX: cells ,PlotInterval:-1,numOfCellsT: cells ,dbPath: dbPath );
                Job j = new Job(tmp_cntrl.SessionName, typeof(XESTSF.XESTSFMain));// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                j.RetryCount = 2;
                j.SetControlStatement($"cs:XESTSF.XESTSFHardCodedControl.AcousticWave1D(waveLength:{waveLength},scaling:{scaling},endTime:{endTime},MachL:{MachL},withLevelSet:{withLS},wavePosition:{wavePos}, shockPosition:{shockPos},p_amp_neg: {amp_neg}, p_amp_pos:{amp_pos},waveform:\"{waveform}\",dgDegree:{p},numOfCellsX:{cells},PlotInterval:-1,numOfCellsT:{cells},dbPath:@\"{dbPath}\")");
                j.NumberOfThreads=1;
                Console.WriteLine("!!!!!!!!!!!!!!!! WARNING: OPENMP Paralelization turned off !!!!!!!!!!!!");
                j.Activate(BoSSSshell.GetDefaultQueue()); 
            }                                                                                                                                                          
        }
    }
    
} } } } }

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(7*86400 , 60); // wait for one week, or until all jobs finish

## Check if residual norms converged

Load sessions

In [ ]:
var sessions = BoSSSshell.WorkflowMgm.DefaultDatabase.Sessions;
sessions

Obtain the residual norms

In [ ]:
var ass= new MatrixAssembler(1);
var res = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).ResHistory.Last());
res

In [ ]:
var ass= new MatrixAssembler(1);
var enres = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).EnResHistory.Last());
enres

Assert if smaller than threshold

In [ ]:
res.ForEach(r => NUnit.Framework.Assert.IsTrue(r<1e-06));
enres.ForEach(r => NUnit.Framework.Assert.IsTrue(r<1e-03));